In [1]:
import os
import pandas as pd

folder_path = "../results/"
transect_df_file = "../data/results_long.csv"

id_vars = [
    'WDPA_PID', 'transectID', 'pointID',
    'max_extent', 'gHM', 'elevation', 'slope'
]

value_vars = [str(y) for y in range(2001, 2022)]

chunksize = 100_000
first = True

all_files = sorted(
    os.path.join(folder_path, f)
    for f in os.listdir(folder_path)
    if f.endswith(".csv")
)

for file in all_files:
    for chunk in pd.read_csv(file, chunksize=chunksize):
        long = chunk.melt(
            id_vars=id_vars,
            value_vars=value_vars,
            var_name='year',
            value_name='value'
        )

        long.to_csv(
            transect_df_file,
            mode="a",
            index=False,
            header=first
        )

        first = False
        del chunk, long

/tmp/ipykernel_13529/2450108534.py:24: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file, chunksize=chunksize):
/tmp/ipykernel_13529/2450108534.py:24: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file, chunksize=chunksize):
/tmp/ipykernel_13529/2450108534.py:24: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file, chunksize=chunksize):
/tmp/ipykernel_13529/2450108534.py:24: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file, chunksize=chunksize):
/tmp/ipykernel_13529/2450108534.py:24: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file, chunksize=chunksize):
/tmp/ipykernel_13529/245010853

In [ ]:
import numpy as np

df = pd.read_csv("..data/results_long.csv")

# pivot gradient values
pts = (
    df
    .pivot_table(
        index=['WDPA_PID', 'year', 'transectID'],
        columns='pointID',
        values='value',
        aggfunc='first'
    )
    .rename(columns={
        2: 'pt_2',
        1: 'pt_1',
        0: 'pt_0',
       -1: 'pt_-1',
       -2: 'pt_-2'
    })
)
pts = pts.dropna(subset=['pt_2', 'pt_1', 'pt_0', 'pt_-1', 'pt_-2'])
covars = (
    df
    .groupby(['WDPA_PID', 'year', 'transectID'])
    .agg(
        trnst_max_extent=('max_extent', 'min'),
        gHM_mean_outer=('gHM', lambda x: x[df.loc[x.index, 'pointID'].isin([1, 2])].mean()),
        elevation_pt0=('elevation', lambda x: x[df.loc[x.index, 'pointID'] == 0].iloc[0]),
        slope_pt0=('slope', lambda x: x[df.loc[x.index, 'pointID'] == 0].iloc[0])
    )
)
transect_df = pts.join(covars)
transect_df['edge'] = (
    (transect_df['pt_0'] > transect_df[['pt_2', 'pt_1']].min(axis=1)) &
    (transect_df['pt_0'] > transect_df[['pt_-2', 'pt_-1']].min(axis=1))
).astype(int)

: 

In [ ]:

def cohens_d(mean1, mean2, std1, std2):
    return (mean1 - mean2) / np.sqrt((std1**2 + std2**2) / 2)

g = transect_df.groupby(['WDPA_PID', 'year'])

wdpa = g.agg(
    n_trnst=('pt_0', 'count'),
    
    pt_2_mean=('pt_2', 'mean'),
    pt_2_std =('pt_2', 'std'),
    pt_1_mean=('pt_1', 'mean'),
    pt_1_std =('pt_1', 'std'),
    pt_0_mean=('pt_0', 'mean'),
    pt_0_std =('pt_0', 'std'),
    pt_-1_mean=('pt_-1', 'mean'),
    pt_-1_std =('pt_-1', 'std'),
    pt_-2_mean=('pt_-2', 'mean'),
    pt_-2_std =('pt_-2', 'std'),

    edge_extent=('edge', 'mean'),
    water_extent=('trnst_has_water', 'mean'),

    gHM_mean=('gHM_mean_outer', 'mean'),
    NDVI_mean=('NDVI', 'mean'),
    elevation_mean=('elevation_pt0', 'mean'),
    slope_mean=('slope_pt0', 'mean')
)

wdpa['D02']  = cohens_d(wdpa.pt_0_mean, wdpa.pt_2_mean,
                        wdpa.pt_0_std,  wdpa.pt_2_std)
wdpa['D01']  = cohens_d(wdpa.pt_0_mean, wdpa.pt_1_mean,
                        wdpa.pt_0_std,  wdpa.pt_1_std)
wdpa['D0-1'] = cohens_d(wdpa.pt_0_mean, wdpa.pt_-1_mean,
                        wdpa.pt_0_std,  wdpa.pt_-1_std)
wdpa['D0-2'] = cohens_d(wdpa.pt_0_mean, wdpa.pt_-2_mean,
                        wdpa.pt_0_std,  wdpa.pt_-2_std)

wdpa['edge_intensity'] = wdpa[['D02', 'D01', 'D0-1', 'D0-2']].min(axis=1)

In [ ]:
attrs = pd.read_csv("attribute_final.csv")[
    ['WDPA_PID', 'PERIMETER', 'AREA_DISSO', 'IUCN_CAT',
     'ORIG_NAME', 'biome', 'STATUS_YR']
]

wdpa = wdpa.reset_index().merge(attrs, on='WDPA_PID', how='left')

In [ ]:
# label wdpa_pids as (edge_extent_result: sig_increase, sig_decrease, no_change, increase, decrease)

# figure 1: methods figure of transect idea
# figure 2: map of edge_extent_result. (join spatial information)
# figure 2a: sub image of 4 examples 2020 magnitude_gradient image, ndvi_image, raw satellite image
# find most edgey ones but for built, vegetation, elevation, water
# figure 3: pool edge_extent_result by biome and replicate fig3b
# figure 4a: edge_extent ~ AREA_DISSO + gHM_mean + ndvi_mean + elevation_mean + slope_mean + water_extent + (1|biome + year)
#           plot this to show relationship between edge_extent and the covariates
# figure 4b: edge_intensity ~ AREA_DISSO + gHM_mean + ndvi_mean + elevation_mean + slope_mean + water_extent + (1|biome + year)
#           plot this to show relationship between edge_intensity and the covariates
# anova for IUCN_CAT, STATUS_YR, biome, AREA_DISSO to check influence
# figure S1: overall distribution of edge_extent and edge_intensity
# figure S2: correlation plot of edge_intensity vs edge_extent

In [ ]:
# total number of unique WDPA_PID
# total number of transects
# total number of unique ISO3
# total number of unique biome
# total number of WDPA_PID per biome
# percent and number of WDPA_PID edge_extent < 10%
# percent and number of WDPA_PID edge_intensity < 0